In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
# print(m/(m+n),'baseline accuracy')
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
# directory = './data/mRisk/Locations.zip'
# output_directory = './data/mRisk/'
# with zipfile.ZipFile(directory, 'r') as zip_ref:
#     zip_ref.extractall(output_directory)
user_indices = ['3016','047dfad3-0807-37f0-bc90-7fd9dedb9bde','3002','1f879d60-3ccc-3b7a-b522-cbfbea277a9d',          '3003','0dd6b3c7-0314-3e6d-ac60-dde239068a6c','3100','7b6446b4-cfe0-3bf7-ae2f-c085363987db',
          '3004','fd053cef-b5a5-3fe6-a158-b6702bce0665','3006','a8ca0b44-85fa-34f4-82d1-30612ac85961',          '3005','bc3e88a3-7bcf-3c61-9284-5886c604e029','3007','9d21b1fc-d2ea-301e-a7b2-ac8a1fd891e1',
          '3008','f30bb859-43dd-37fb-b4ce-18ae832a5b55','3009','8ae05753-f5ed-3a3e-90a5-f96e991ffe38',          '3010','cb064c00-5517-3828-8309-a2bfe8310add','3011','f2e77c23-06ef-3ae0-909b-36c5e1a25b6e',
          '3012','d06e1611-fdd0-30b9-a8ab-252e1d68472a','3013','da7f5eb4-0239-3303-ae48-b5e91d6dfdd9',          '3014','d2b0b3cd-3fbe-3c2e-9677-1af7de9309ff','3015','7d7b93e0-cf67-37b7-9df9-42ebac7dbb07',
          '3017','1466e642-d0f4-368b-8998-2b5894d4b0f0',          '3018','7fab23f1-9c50-345c-837e-e020482d7e0d','3019','d1165f4c-ae82-308f-b0fe-66789c1d2d1b',          '3020','558d0012-7c68-38af-8194-bbd71bac5152','3022','b4bcc0ff-b0fa-3996-93c6-37ec7957321a',
          '3023','7cd86bed-89d0-31f3-b230-16ae51614579','3024','9e34ce7f-a634-3614-88e2-659c5e5aa3ef',          '3025','4c6c5b43-21ec-3c74-9603-7d808574e487','3026','36156e40-77d1-3827-b6af-bafb5b2aeb19',
          '3027','ca6b9b66-7af3-3b7e-b89e-1c2ce17cfd07','3028','daddce87-4f77-397f-8f26-0c4251d68080',          '3029','66b8a9a9-7db7-3fdc-b3b1-5ad52706a758','3030','df5ce7f6-d61f-33a6-b3bf-94e2f97ddc0c',
          '3031','fe6a5545-ca7b-38e2-aa0d-021c5e675f15','3032','0c155e6b-410c-329e-8a06-66d01424ad53',          '3033','dd3e9707-8d11-35ff-8c4a-f7d62fafc6a3','3034','bdc04055-5495-3233-beed-de7b486888aa',
          '3035','79a084d6-94c8-3041-8699-fe6d2a4be58c','3036','3467c5f3-6986-3421-a805-0473035df5cf',          '3037','a2dc45f9-ecc9-35b5-bdfd-0e282bd2fafe','3038','4a757428-0041-3310-89cf-29bcc95a3585',
          '3039','ee1c54b3-ed15-3755-a53d-a381f62a305b','3040','3da2c05f-b2d2-3189-ab4a-42aaea76155e',          '3041','73469433-515d-3fbf-87a0-713716d271e9','3042','8917070a-fc51-394f-820e-b7d0331de03d',
          '3043','ab5aa320-fb07-3e2d-a5dc-75b913a5f944','3044','65c8982a-97ab-37ca-bedf-db22e67aeedd',          '3045','58fe1dd7-e73f-359d-8b75-60048b6dda3f','3046','3bff28ff-eb12-3d3f-897d-564a2d4bb38d',
          '3047','fdf6d51e-ad75-35d2-aaae-889cf4e4e0bc','3048','6c1a560d-85da-3c3b-b655-c46592e70406',          '3049','ac3db1e1-a0d7-3722-b674-cc51e42da6e3','3050','779db0c5-d56c-3cf9-8d2f-61ff39dec4bb',
          '3051','ce905877-c2eb-3450-823f-626c658c8d7f','3052','a29eb24d-d576-3130-b646-187dc83bba42',          '3053','2e338495-136f-3d74-9891-1c7c3541fa28','3054','5ab7a1f0-532f-3404-96fc-5fb16803fca8',
          '3055','07689307-0253-3d2a-a975-bdf3b442e2b0','3056','476efab6-7333-3246-8152-06e03cd10850',          '3057','3fd0db3c-ae87-351a-a343-402548a4b360','3058','f4e74737-697d-3635-855d-35d1375b704a',
          '3059','9e515982-1fd1-3d1b-a735-2d79b4fd5352','3060','c443e037-70d0-3b09-a1df-12298c4875dc',          '3061','741f02af-3de7-355c-a699-1f22cec5e8bf','3062','00a0bf3a-b0d9-3b02-9ed3-8afbfb6e2730',
          '3063','864d3e4f-c69f-3b32-a785-7a84a1d401ae','3064','719eeb3e-c78c-3e3a-898f-12f3cc4525d0',          '3065','ed34b7ed-98fd-33f1-ad8a-d5dd46e6dcb0','3066','659a345f-351c-3989-8174-8e2f0632e861',
          '3067','d5672d19-ff58-3c95-980e-aa8d350ddbeb','3068','243ecad5-923a-3388-a99b-dc14c700cd29',          '3069','b09df1fe-acb7-3da3-ba73-c9ec8951052f','3070','a771acda-75a2-3d2c-a0a8-e153f821a792',
          '3072','c8e60103-cc9b-3fee-bbce-778ed57afcec','3073','b116dc86-34e1-34b5-aa25-57b1cb7f0ea0',          '3074','071ece63-5dda-344a-8e52-1fc7ddf357bb','3075','aebdd8bb-c236-3931-a88b-f4e890f88c33',
          '3076','4d2549d4-c6b2-3243-a8b6-9fe36dcb0a66','3077','8e387766-4b0a-3a04-807c-d0419b4a0497',          '3078','deee7031-3cf0-3e97-9728-4747104da59b','3079','0a54a64b-b788-3b75-8cbe-84c5cf15bb8e',
          '3080','440b6766-44c3-358a-966f-293e5e1c66b3','3081','8ee395ae-3641-3542-bc82-c4e2125e3cc1',          '3082','c12398b1-7cb3-3e06-8b7d-4bb2aa87e875','3083','cd55ae15-ee0e-3527-b7f5-d5a4897d5b8e',
          '3084','ee05a9e3-cef7-3b71-8960-cd493eb64708','3085','9c06cc8a-0603-3a46-907d-dd63a8f2bd99',          '3086','af642ecf-911b-31bb-8873-700ccc441680','3087','cdac2efd-c877-34cd-aa08-7470a464c40d',
          '3089','c2e0dc7c-ff9c-3ee0-8f7f-442c4c3f6bb0','3088','082f7d1e-f7e1-3cbe-8dc7-c81a101380b7',          '3090','0cb209c9-886f-3358-91c6-b7625552b813','3091','6a551303-9f23-30c3-9fab-b549933923bd',
          '3092','9df62dc7-bed0-3e3a-a662-de8dcda5c5e4','3093','8760670c-cb61-33ac-896e-4c83c58ed9a0',          '3094','2e1a66cc-9b47-3500-ae0c-862910d9595f','3095','2b783fcd-e6db-3dc1-b065-4e409d356716',
          '3096','faa2df18-dad3-373e-a463-000fa352ea21','3097','a29bd5f3-0774-3072-b6f8-3e141d4ceab2',          '3098','b641d1c8-d3ab-3d9b-bb6b-ffb273b972cf','3099','5415885c-20f6-3fd9-8d6a-23914db7a633',
          '3101','6e6d7a35-d031-3eb2-be41-97d176ff3552','3102','76e4a4a6-4025-3757-9886-3ef05c2a224f',          '3103','79fdf14f-d494-3036-953b-628eec84b8cb','3104','bdfe86ca-f760-32c9-93ff-6b611fa87b0f',
          '3105','aca62619-56dc-3444-9bdb-beb77b7239b2','3106','151bafe0-483e-33d5-98aa-e666c19ae219',          '3107','7f0d43c5-a9cb-35ce-862a-39a475f15bf5','3109','0e996cd5-488d-3652-9c9a-dac934a5a8a7',
          '3108','6d10fd72-39c8-3ff9-813f-058e924de0f3','3110','1a55c60f-0a68-3fff-bcf5-94dab5b8dd76',          '3111','2852e4e5-59f4-3873-aae8-b3a6a461cde2','3112','51009acd-9b5f-3dc6-805a-7435bc931a62',
          '3113','a26bcfe4-4bdf-30b3-b592-6eb85bf33771','3114','550a38b3-d4b6-37ff-9597-cbe473631ed3',          '3115','513cdafb-5c26-39ba-ba24-33e597bea347','3116','88106351-980e-3b86-8b19-6f62956b48c8',
          '3117','526ce985-4b5e-3b63-b44c-255a1931dc88','3118','8e8cbfd3-27d1-3483-89d0-141565906ba1',          '3119','bc210f41-d98b-370f-9c5c-dc12d5cb73e9','3121','ca9e0b25-477b-3bff-a4d4-a4395afd3e38',
          '3120','131dfd9a-0816-370e-a1f7-44de0eace4fd','3122','24770270-32bf-3229-8343-08cdc7fa8246',          '3123','159a06ce-a78d-32a3-aee5-d0cc9619fc7a','3124','d788cd10-b6bb-3372-a781-74373e6e34f6',
          '3125','db2d1c42-040a-3fc1-866d-12fe25d4a532','3126','946ac096-5c75-32b6-900f-3c66551ab94e',          '3127','e87fe51c-5d01-37dd-917a-ac91b80a2199','3128','0a967d9d-c5dd-3969-a50d-4428ffbad7d4',
          '3129','d3c9d61c-1d46-3e0a-89a9-2aff362c09fb','3130','565b5348-ece9-3cd8-bd7e-1f6864a28289',          '3131','278c9be6-086c-3a29-b528-88b8a9238d25','3133','58e8e0af-5d3a-32d4-81b6-6c8d7ea91475',
          '3134','e9c99c80-f79d-32d6-bfc8-54b9c8ce9605','3132','b21f3d4c-8429-397b-a773-c5e850705008',          '3135','4c530d1e-4997-37b5-9dec-c810836c78a4','3136','eb0ad6f3-143b-382d-8850-b938334d5dee',
          '3137','7aeb3dfb-e659-3fd8-8cfd-aa6a114cee02','3138','8eaa7ae7-2f1b-3ec4-a2f2-8a21f622e550',          '3139','1f281d5e-d82d-31b6-938c-34e0129c138a','3140','8aa9a1af-e9e0-39b8-9dd0-0147fc3e095b',
          '3141','be3a8e70-2722-302e-ad2a-69d74863d44b','3142','b5dccf0b-86f2-3c02-87a6-5f7834957b34',          '3143','b9da40a7-8ee6-32c4-a3a2-bdfc91b269ec','3144','e149328d-d4ae-37f4-8bf0-357fddad718d',
          '3145','c52ce7ff-15bc-37c0-b5ce-c5c19ae552db','3146','40f4bb48-e0af-32f7-b48d-e4016f52d912',          '3147','0a589a7d-f4cd-3eb5-bea1-0bd42dd0f107',          '3148','2bc12753-c469-3121-ab01-cc224064b81f','3149','969cfa6c-c5bb-3ce8-970e-4bf2fd279d66',          '3150','68634b51-0c55-3d43-b0eb-6553140a6572','3151','bc1941e9-4ac9-358f-9c2c-ae09c7b2ddbd',
          '3152','897337e9-5cc4-3e1d-b62e-6a5a696f7bbf','3153','40cf2f37-51f5-3066-b248-f32d1f6256b6',          '3154','85c582ce-ea47-3a0c-975e-1d4921a3ecae','3155','8d9a4db6-4769-382f-9acc-41a3f7916746',
          '3156','4eb58fe4-452f-3c98-aa43-9b5384ca448a','3157','507931ad-e283-3ee2-a173-ce37a3b09877',          '3158','f4f201c1-8224-3b09-8884-be5ba9de7c13','3159','f85220b4-80b7-3482-a5c7-48a450691060',
          '3160','2dc79999-bcdc-3545-b095-7189ab110aee','3161','a76a9573-dcc2-35a6-91a1-e3e8d7e26f59',          '3162','b9f69412-4451-3494-899f-1b4e391c5d30', '3163','fb6d63e0-c0e4-351b-b496-96d1d5ecf2ef',
          '3164','6cc79dec-86b7-326f-8014-53d3dc64fad8','3165','2bab39b1-2833-3c21-a296-488d4e0e1c1f',          '3166','19e0f55c-a224-3a31-9a92-59de3574f8fb','3167','40d34ca7-cdf0-39d6-889a-f66fd41fda94',
          '3168','2053bad6-739f-3ede-8f77-ccb51739d122','3169','8b6dcb23-fc7e-355d-b86d-6cf08a556706',          '3170','4e777422-0b3b-31ac-bb30-4f4764423c1a','3171','d5bd7686-2445-30d5-be50-609708d58612']

user_dict_reverse = {user_indices[i]:user_indices[i+1] for i in range(0,len(user_indices)-1,2)}

user_dict = {user_indices[i+1]:user_indices[i] for i in range(0,len(user_indices)-1,2)}

def fix_timestamp(a):
    start_daylight = pd.Timestamp(a.year,3,14,0,0,0)
    end_daylight = pd.Timestamp(a.year,11,7,0,0,0)
    if start_daylight<=a<=end_daylight:
        return a+timedelta(hours=1)
    return a

def get_features(a):
    return [np.percentile(a,90),np.percentile(a,95),np.percentile(a,80),np.percentile(a,5),np.percentile(a,10),np.percentile(a,20),
            np.percentile(a,90) - np.percentile(a,80),np.percentile(a,20) - np.percentile(a,10),
            np.percentile(a,90) - np.percentile(a,10),np.percentile(a,80) - np.percentile(a,20),
            np.median(a),iqr(a),skew(a)]


def get_time_and_instantaneous_features(df):
    df = df.sort_values('window').reset_index(drop=True)
    df['localtime_fixed'] = df['localtime'].apply(lambda a:fix_timestamp(a))
    all_data = []
    start_time = df['localtime_fixed'].iloc[0]
    start_index = 0
    if df['user'].values[0] in user_dict:
        user_id = user_dict[df['user'].values[0]]
    else:
        user_id = df['user'].values[0]
    day = df['day'].values[0]
    for i,row in df.iterrows():
        if row['localtime_fixed']-start_time<=timedelta(minutes=obs):
            continue
        temp_data = df['stress_probability_smooth'].iloc[start_index:i].values
        if len(temp_data)<50:
            continue
        feature = [user_id,day,start_time+timedelta(minutes=obs//2),start_time,
                   start_time+timedelta(minutes=obs)]
        feature_raw=get_features(temp_data)
        feature_raw+=get_features(np.diff(temp_data))
        feature.append(np.array(feature_raw))
        daily_features = get_features(df['stress_probability_smooth'].iloc[:i].values)
        feature.append(daily_features)
        all_data.append(feature)
        start_time+=timedelta(minutes=sliding)
        while start_index<df.shape[0] and df['localtime_fixed'].iloc[start_index]<start_time:
            start_index+=1
    return pd.DataFrame(all_data,columns=['user','day','time','start_time','end_time','feature','daily'])

from math import *
times = {}
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6373 * c
    return float(km)*1000

def get_smoking_status(df):
    
    user_id = df.user.values[0]
    if not os.path.isfile(location_data_directory+user_id+'_GPS_DATA.csv') or \
    not os.path.isfile(data_directory+'SmokingSpotStatus/'+user_id+'_Smoking_Spot_Status.csv'):
        return pd.DataFrame([],columns=list(df.columns)+['is_smoking','spread','distance_to_nearest_spot','Latitude','Longitude'])
    df = df.sort_values('start_time').reset_index(drop=True)
    location_data = pd.read_csv(location_data_directory+user_id+'_GPS_DATA.csv')
    smoking_spots = pd.read_csv(data_directory+'SmokingSpotStatus/'+user_id+'_Smoking_Spot_Status.csv')
    location_data['start_time'] = location_data['StartTimeUnix'].apply(lambda a:datetime.fromtimestamp(a/1000))
    visit_data = pd.read_csv(data_directory+'SmokingSpotVisitationHistory/'+user_id+'_SmokingSpot_Visitation_History.csv')
    visit_data['Duration_of_stay'] = visit_data['Duration_of_stay (minutes)'].apply(lambda a:a)
    visit_data['Arrival_Time'] = visit_data['Arrival_UTimestamp'].apply(lambda a:datetime.fromtimestamp(a/1000))
    visit_data['Departure_Time'] = visit_data['Departure_UTimestamp'].apply(lambda a:datetime.fromtimestamp(a/1000))
    df['Latitude'] = [0.0]*df.shape[0]
    df['Longitude'] = [0.0]*df.shape[0]
    df['is_smoking'] = [-1.0]*df.shape[0]
    df['spread'] = [-1.0]*df.shape[0]
    df['distance_to_nearest_spot'] = [-1.0]*df.shape[0]
    df['time_spent_in_transition'] = [-1.0]*df.shape[0]
    df['time_spent_in_smoking_spot'] = [-1.0]*df.shape[0]  
    for i,row in df.iterrows():
        temp = location_data[(location_data.start_time<row['end_time']) & (location_data.start_time>=row['start_time'])]
        if temp.shape[0]<5:
            continue
        mean_lat = np.average(temp['Latitude'],weights=temp['Accuracy'])
        mean_long = np.average(temp['Longitude'],weights=temp['Accuracy'])
        temp['distance'] = temp.apply(lambda a:haversine(a['Latitude'],a['Longitude'],mean_lat,mean_long),axis=1)
        spread = temp['distance'].mean()
        df['Latitude'].at[i] = mean_lat
        df['Longitude'].at[i] = mean_long
        smoking_spots['distance'] = smoking_spots.apply(lambda a:haversine(a['Latitude'],a['Longitude'],mean_lat,mean_long),axis=1)
        distance_categories = smoking_spots[['Smoking_Spot_Characterization','distance']].values
        index = distance_categories[:,1].argmin()
        distance_to_spot = distance_categories[index,1]
        is_smoking = int(distance_categories[index,0].lower() in ['general','personal'])
        df['is_smoking'].at[i] = is_smoking
        df['spread'].at[i] = spread
        df['distance_to_nearest_spot'].at[i] = distance_to_spot
        start_time = row['start_time']
        end_time = row['end_time']
        temp_visit_data = visit_data[(visit_data.Arrival_Time<=end_time)&(visit_data.Departure_Time>=start_time)].sort_values('Arrival_Time').reset_index(drop=True)
        if temp_visit_data.shape[0]>0:
            ii = temp_visit_data.shape[0]-1
            if temp_visit_data.iloc[ii]['Departure_Time']>end_time:
                temp_visit_data['Departure_Time'].at[ii] = end_time
                temp_visit_data['Duration_of_stay'].at[ii] = (temp_visit_data['Departure_Time'].at[ii]-temp_visit_data['Arrival_Time'].at[ii]).total_seconds()/60
            if temp_visit_data.iloc[0]['Arrival_Time']<start_time:
                temp_visit_data['Arrival_Time'].at[0] = start_time
                temp_visit_data['Duration_of_stay'].at[0] = (temp_visit_data['Departure_Time'].at[0]-temp_visit_data['Arrival_Time'].at[0]).total_seconds()/60
            total = temp_visit_data['Duration_of_stay'].sum()
            temp_visit_data_smoking = temp_visit_data[temp_visit_data.Semantic.isin(['Alcohol','Smoke','Retail'])]
            temp_visit_data_transit = temp_visit_data[temp_visit_data.Semantic.isin(['Transit'])]
            if temp_visit_data_transit.shape[0]>0:
                df['time_spent_in_transition'].at[i] = temp_visit_data_transit['Duration_of_stay'].sum()/total
            else:
                df['time_spent_in_transition'].at[i] = 0
            if temp_visit_data_smoking.shape[0]>0:
                df['time_spent_in_smoking_spot'].at[i] = temp_visit_data_smoking['Duration_of_stay'].sum()/total            
            else:
                df['time_spent_in_smoking_spot'].at[i] = 0
            
        else:
            df['time_spent_in_transition'].at[i] = 0
            df['time_spent_in_smoking_spot'].at[i] = 0

    df['location_features'] = df.apply(lambda a:[a['is_smoking'],a['spread'],
                                        a['distance_to_nearest_spot'],
                                        a['time_spent_in_transition'],
                                        a['time_spent_in_smoking_spot']],axis=1)

    return df


def get_smoking_event_features(df):
    user_id = df['user'].values[0]
    df = df.sort_values('start_time').reset_index(drop=True)
    visit_data = pd.read_csv(data_directory+'SmokingSpotVisitationHistory/'+user_id+'_SmokingSpot_Visitation_History.csv')
    visit_data['Duration_of_stay'] = visit_data['Duration_of_stay (minutes)'].apply(lambda a:a)
    visit_data['Arrival_Time'] = visit_data['Arrival_UTimestamp'].apply(lambda a:datetime.fromtimestamp(a/1000))
    visit_data['Departure_Time'] = visit_data['Departure_UTimestamp'].apply(lambda a:datetime.fromtimestamp(a/1000))
    start_time_all = df['start_time'].iloc[0]-timedelta(hours=48)
    visit_data = visit_data[visit_data.Arrival_Time>=start_time_all]
    df['s1'] = [1.0]*df.shape[0]
    df['s2'] = [1.0]*df.shape[0]
    df['s3'] = [1.0]*df.shape[0]
    smoking_episode_features = []
    for i,row in df.iterrows():
        start_time = row['start_time']
        end_time = row['end_time']
        temp_visit_data = visit_data[visit_data.Arrival_Time<=end_time].sort_values('Arrival_Time').reset_index(drop=True)
        ii = temp_visit_data.shape[0]-1
        if temp_visit_data.shape[0]>0 and temp_visit_data.iloc[temp_visit_data.shape[0]-1]['Departure_Time']>end_time:
            temp_visit_data['Departure_Time'].at[ii] = end_time
            temp_visit_data['Duration_of_stay'].at[ii] = (temp_visit_data['Departure_Time'].at[ii]-temp_visit_data['Arrival_Time'].at[ii]).total_seconds()/60

        temp_visit_data_smoking = temp_visit_data[temp_visit_data.Semantic.isin(['Alcohol','Smoke','Retail'])].sort_values('Arrival_Time').reset_index(drop=True)
        smoking_episode_this_window = np.zeros((obs,3))
        smoking_episode_this_window[:,1] = 1
        
        if temp_visit_data_smoking.shape[0]>0:
            time_since_last_visit_smoking_spot = max(0,(start_time-temp_visit_data_smoking.iloc[temp_visit_data_smoking.shape[0]-1]['Departure_Time']).total_seconds()//60)
            duration_of_visit = temp_visit_data_smoking.iloc[temp_visit_data_smoking.shape[0]-1]['Duration_of_stay']
            fraction_of_minutes_spent_in_smoking_spots_out_of_observed = temp_visit_data_smoking['Duration_of_stay'].sum()/temp_visit_data['Duration_of_stay'].sum()
            smoking_episode_this_window[:,2] = duration_of_visit
            smoking_episode_this_window[:,0] = np.arange(time_since_last_visit_smoking_spot,time_since_last_visit_smoking_spot+obs,1)
        else:
            time_since_last_visit_smoking_spot = (start_time-start_time_all).total_seconds()//60
            duration_of_visit = 0
            fraction_of_minutes_spent_in_smoking_spots_out_of_observed = 0
            smoking_episode_this_window[:,1:] = 0
            smoking_episode_this_window[:,0] = np.arange(time_since_last_visit_smoking_spot,time_since_last_visit_smoking_spot+obs,1)
        smoking_episode_features.append(np.expand_dims(smoking_episode_this_window,axis=0))            
        df['s1'].at[i] = time_since_last_visit_smoking_spot
        df['s2'].at[i] = duration_of_visit
        df['s3'].at[i] = fraction_of_minutes_spent_in_smoking_spots_out_of_observed
    df['smoking_event_features'] = df.apply(lambda a:[a['s1'],a['s2'],a['s3']],axis=1)
    df['smoking_episode'] = list(np.concatenate(smoking_episode_features))
    return df


def get_labels(df):
    user_id = int(df['user'].values[0])
    lapse_data['Participant'] = lapse_data['Participant'].apply(lambda a:int(a))
    temp = lapse_data[lapse_data.Participant==user_id].sort_values('Lapse').reset_index(drop=True)
    if temp.shape[0]==0:
        return pd.DataFrame([],columns=list(df.columns)+['Label','time_since_quit','quit_episode'])
    
    temp['quit_time'] = temp['Quit Day'].apply(lambda a:datetime.fromtimestamp(a/1000))
    quit_time = temp['quit_time'].values[0]
    df = df[df.start_time>=quit_time]
    df = df.sort_values('time').reset_index(drop=True)
    df['Label'] = [0]*df.shape[0]
    df['time_since_quit'] = [-1]*df.shape[0]
    df['quit_episode'] = [np.zeros((obs,3))]*df.shape[0]
    quit_features = []
    if len(temp['Lapse'].values)>0:
        temp['lapse_time'] = temp['Lapse'].apply(lambda a:datetime.fromtimestamp(a/1000))
        lapse_times = temp['lapse_time'].values
        final_index = 0
        df = df.sort_values('time').reset_index(drop=True)
        for i,row in df.iterrows():
            st = row['start_time']
            et = row['end_time']
            for lapse_time in lapse_times:
                if et<=lapse_time<=et+timedelta(minutes=(intervention_window+prediction_window)):
                    df['Label'].at[i] = 1
                    final_index = max(final_index,i)
            time_since_quit = (st-quit_time).total_seconds()//60
            df['time_since_quit'].at[i] = time_since_quit
            this_window_features = np.zeros((obs,3))
            this_window_features[:,0] = np.arange(time_since_quit,time_since_quit+obs,1)
            this_window_features[:,1] = 1
            quit_features.append(np.expand_dims(this_window_features,axis=0))
        if final_index>0:
            df['quit_episode'] = list(np.concatenate(quit_features))
            return df.iloc[:(final_index+1)]
    else:
        return pd.DataFrame([],columns=list(df.columns))

def get_activity_episode_encoding(df,max_episodes = 10,min_activity_episode_duration=1):
    df = df.sort_values('time').reset_index(drop=True)
    df_activity = activity_data[(activity_data.user==df.user.values[0])&(activity_data.day==df.day.values[0])]
    df_activity['episode_duration'] = df_activity.apply(lambda row:(row['end']-row['start']).total_seconds()/60,axis=1)
    df_activity = df_activity[(df_activity['episode_duration']>min_activity_episode_duration)&(df_activity.episode_label=='NON STATIONERY')]
    df_activity = df_activity.sort_values('start').reset_index(drop=True)
    if df.shape[0]<30:
        return pd.DataFrame([],columns=list(df.columns)+['activity_episode'])
    if df_activity.shape[0] > 0:
        start_time = min(df_activity['start'].iloc[0],df_activity['start'].iloc[0],df['start_time'].iloc[0]-timedelta(minutes=1))
        end_time = df['end_time'].iloc[df.shape[0]-1]
    else:
        start_time = df['end_time'].iloc[0]
        end_time = df['end_time'].iloc[df.shape[0]-1]
    all_data = []
    user_id = df.user.values[0]
    day = df.day.values[0]
    activity_features = []
    for i,row in df.iterrows():
        start_ts = row['start_time']
        end_ts = row['end_time']
        episodes_all = df_activity[(df_activity.start<end_ts)&(df_activity.episode_label=='NON STATIONERY')].sort_values('start').reset_index(drop=True)
        if episodes_all.shape[0]==0:
            activity_features.append(np.zeros((1,max_episodes,obs,3)))
            continue
        activity_features_this_window = []
        for j,episode in episodes_all.iterrows():
            amplitude = 1
            duration = (episode['end']-episode['start']).total_seconds()/60
            episode_features = np.zeros((obs,3))
            episode_features[:,1] = amplitude ## amplitude 
            episode_features[:,2] = duration ## duration
            mid = episode['start']+timedelta(seconds = duration/2)
            if episode['end']<=start_ts:
                episode_to_start_time_diff = int((start_ts-mid).total_seconds()//60)
                episode_features[:,0] = np.arange(episode_to_start_time_diff,episode_to_start_time_diff+obs,1)
            elif episode['start']<=start_ts and episode['end']<=end_ts:
                index_of_end = int((episode['end']-start_ts).total_seconds()//60)
                episode_features[:index_of_end,0] = 0
                episode_features[index_of_end:,0] = np.arange(0,obs-index_of_end,1)
            elif episode['start']>start_ts and episode['end']<=end_ts:
                index_of_start = int((episode['start']-start_ts).total_seconds()//60)
                episode_features[:index_of_start,0] = 0
                episode_features[:index_of_start,1:] = 0
                index_of_end = int((episode['end']-start_ts).total_seconds()//60)
                episode_features[index_of_start:index_of_end,0] = 0
                episode_features[index_of_start:index_of_end,1] = amplitude
                episode_features[index_of_start:index_of_end,2] = duration
                episode_features[index_of_end:,0] = np.arange(0,obs-index_of_end,1)
            elif episode['start']<=end_ts and episode['end']>end_ts:
                index_of_start = int((episode['start']-start_ts).total_seconds()//60)
                episode_features[:index_of_start,0] = 0
                episode_features[:index_of_start,1:] = 0
                episode_features[index_of_start:,0] = 0
                episode_features[index_of_start:,1] = amplitude 
                episode_features[index_of_start:,2] = duration
            activity_features_this_window.append(episode_features.reshape(1,obs,3))
        activity_features_this_window = np.concatenate(activity_features_this_window)
        if activity_features_this_window.shape[0]>=max_episodes:
            activity_features_this_window = activity_features_this_window[-1*max_episodes:,:,:]
        else:
            zero_episodes = max_episodes - activity_features_this_window.shape[0]
            activity_features_this_window = np.concatenate([activity_features_this_window,np.zeros((zero_episodes,obs,3))])
        activity_features.append(np.expand_dims(activity_features_this_window, axis=0))
    df['activity_episode'] = list(np.concatenate(activity_features))
    return df


def get_stress_episode_encoding(df,max_episodes = 10,min_activity_episode_duration=1):
    df = df.sort_values('time').reset_index(drop=True)
    df_activity = stress_episode_data[(stress_episode_data.user==df.user.values[0])&(stress_episode_data.day==df.day.values[0])]
    df_activity['episode_duration'] = df_activity.apply(lambda row:(row['end']-row['start']).total_seconds()/60,axis=1)
    df_activity = df_activity[(df_activity['episode_duration']>min_activity_episode_duration)&(df_activity.stress_episode_label=='STRESS')&(df_activity['imputed_percentage']<.5)]
    df_activity = df_activity.sort_values('start').reset_index(drop=True)
    if df.shape[0]<30:
        return pd.DataFrame([],columns=list(df.columns)+['activity_episode'])
    if df_activity.shape[0] > 0:
        start_time = min(df_activity['start'].iloc[0],df_activity['start'].iloc[0],df['start_time'].iloc[0]-timedelta(minutes=1))
        end_time = df['end_time'].iloc[df.shape[0]-1]
    else:
        start_time = df['end_time'].iloc[0]
        end_time = df['end_time'].iloc[df.shape[0]-1]
    all_data = []
    user_id = df.user.values[0]
    day = df.day.values[0]
    activity_features = []
    stress_seen_or_not = []
    for i,row in df.iterrows():
        start_ts = row['start_time']
        end_ts = row['end_time']
        episodes_all = df_activity[(df_activity.start<end_ts)].sort_values('start').reset_index(drop=True)
        if episodes_all.shape[0]==0:
            activity_features.append(np.zeros((1,max_episodes,obs,3)))
            stress_seen_or_not.append(0)
            continue
        stress_seen_or_not.append(1)
        activity_features_this_window = []
        for j,episode in episodes_all.iterrows():
            amplitude = episode['stress_density']
            duration = (episode['end']-episode['start']).total_seconds()/60
            episode_features = np.zeros((obs,3))
            episode_features[:,1] = amplitude ## amplitude 
            episode_features[:,2] = duration ## duration
            mid = episode['start']+timedelta(seconds = duration/2)
            if episode['end']<=start_ts:
                episode_to_start_time_diff = int((start_ts-mid).total_seconds()//60)
                episode_features[:,0] = np.arange(episode_to_start_time_diff,episode_to_start_time_diff+obs,1)
            elif episode['start']<=start_ts and episode['end']<=end_ts:
                index_of_end = int((episode['end']-start_ts).total_seconds()//60)
                episode_features[:index_of_end,0] = 0
                episode_features[index_of_end:,0] = np.arange(0,obs-index_of_end,1)
            elif episode['start']>start_ts and episode['end']<=end_ts:
                index_of_start = int((episode['start']-start_ts).total_seconds()//60)
                episode_features[:index_of_start,0] = 0
                episode_features[:index_of_start,1:] = 0
                index_of_end = int((episode['end']-start_ts).total_seconds()//60)
                episode_features[index_of_start:index_of_end,0] = 0
                episode_features[index_of_start:index_of_end,1] = amplitude
                episode_features[index_of_start:index_of_end,2] = duration
                episode_features[index_of_end:,0] = np.arange(0,obs-index_of_end,1)
            elif episode['start']<=end_ts and episode['end']>end_ts:
                index_of_start = int((episode['start']-start_ts).total_seconds()//60)
                episode_features[:index_of_start,0] = 0
                episode_features[:index_of_start,1:] = 0
                episode_features[index_of_start:,0] = 0
                episode_features[index_of_start:,1] = amplitude 
                episode_features[index_of_start:,2] = duration
            activity_features_this_window.append(episode_features.reshape(1,obs,3))
        activity_features_this_window = np.concatenate(activity_features_this_window)
        if activity_features_this_window.shape[0]>=max_episodes:
            activity_features_this_window = activity_features_this_window[-1*max_episodes:,:,:]
        else:
            zero_episodes = max_episodes - activity_features_this_window.shape[0]
            activity_features_this_window = np.concatenate([activity_features_this_window,np.zeros((zero_episodes,obs,3))])
        activity_features.append(np.expand_dims(activity_features_this_window, axis=0))
    df['stress_episode'] = list(np.concatenate(activity_features))
    df['stress_seen_so_far'] = stress_seen_or_not
    return df

def get_activity_features_instance(row,df_activity,start_time,end_time):
    start_ts = row['start_time']
    end_ts = row['end_time']

    episodes_before = df_activity[(df_activity.end<start_ts)&(df_activity.episode_label=='NON STATIONERY')].sort_values('start').reset_index(drop=True)

    episodes_within = df_activity[(df_activity.end>start_ts)&(df_activity.start<=end_ts)&(df_activity.episode_label=='NON STATIONERY')].sort_values('start').reset_index(drop=True)

    if episodes_within.shape[0]>0:
        if episodes_within['end'].iloc[episodes_within.shape[0]-1]>end_ts:
            episodes_within['end'].at[episodes_within.shape[0]-1] = end_ts
            episodes_within['episode_duration'].at[episodes_within.shape[0]-1] = (end_ts-episodes_within['start'].at[episodes_within.shape[0]-1]).total_seconds()/60
        percentage_of_active = episodes_within.episode_duration.sum()/obs
        maximum_duration_of_current_activity_episode = episodes_within.episode_duration.max()
        average_duration_of_current_window_episode = episodes_within.episode_duration.mean()
    else:
        percentage_of_active = 0
        maximum_duration_of_current_activity_episode = 0
        average_duration_of_current_window_episode = 0

    if episodes_before.shape[0]>0:
        time_since_last_activity = (start_ts - episodes_before['end'].iloc[episodes_before.shape[0]-1]).total_seconds()/60
        no_activity_till_now = 0
        duration_of_last_activity_episode = episodes_before['episode_duration'].iloc[episodes_before.shape[0]-1]
        average_duration_of_before_activity_episode = episodes_before['episode_duration'].mean()
        percentage_of_active_before = episodes_before['episode_duration'].sum()/((start_ts-start_time).total_seconds()/60)
    else:
        time_since_last_activity = 0
        no_activity_till_now = 1
        duration_of_last_activity_episode = 0
        average_duration_of_before_activity_episode = 0
        percentage_of_active_before = 0

    activity_features = [percentage_of_active,maximum_duration_of_current_activity_episode,average_duration_of_current_window_episode,
                         no_activity_till_now,time_since_last_activity,duration_of_last_activity_episode,average_duration_of_before_activity_episode,
                         percentage_of_active_before]
    return activity_features

def get_stress_features_instance(row,df_stress,start_time,end_time):
    start_ts = row['start_time']
    end_ts = row['end_time']

    episodes_before = df_stress[(df_stress.end<start_ts)&(df_stress.episode_label=='STRESS')].sort_values('start').reset_index(drop=True)
    episodes_within = df_stress[(df_stress.end>start_ts)&(df_stress.start<=end_ts)&(df_stress.episode_label=='STRESS')].sort_values('start').reset_index(drop=True)

    if episodes_before.shape[0]>0:
        no_stress_till_now = 0
        time_since_last_stress = (start_ts - episodes_before['end'].iloc[episodes_before.shape[0]-1]).total_seconds()/60
        duration_of_last_stress_episode = episodes_before['episode_duration'].iloc[episodes_before.shape[0]-1]
        average_duration_of_before_stress_episode = episodes_before['episode_duration'].mean()
        density_of_last_stress_episode = episodes_before['stress_density'].iloc[episodes_before.shape[0]-1]
        average_density_of_before_stress_episode = episodes_before['stress_density'].mean()
        deviation_to_daily_mean_of_last_stress_episode = episodes_before['deviation_to_daily_mean'].iloc[episodes_before.shape[0]-1]
        percentage_of_stress_before = episodes_before['episode_duration'].sum()/((start_ts-start_time).total_seconds()/60)
    else:
        no_stress_till_now = 1
        time_since_last_stress = 0
        duration_of_last_stress_episode = 0
        average_duration_of_before_stress_episode = 0
        density_of_last_stress_episode = 0
        average_density_of_before_stress_episode = 0
        deviation_to_daily_mean_of_last_stress_episode = 0
        percentage_of_stress_before = 0

    if episodes_within.shape[0]>0:
        if episodes_within['end'].iloc[episodes_within.shape[0]-1]>end_ts:
            episodes_within['end'].at[episodes_within.shape[0]-1] = end_ts
            episodes_within['episode_duration'].at[episodes_within.shape[0]-1] = (end_ts-episodes_within['start'].at[episodes_within.shape[0]-1]).total_seconds()/60
        percentage_of_stress = episodes_within.episode_duration.sum()/obs
        maximum_duration_of_current_stress_episode = episodes_within.episode_duration.max()
        average_duration_of_current_stress_episode = episodes_within.episode_duration.mean()
        maximum_density_of_current_stress_episode = episodes_within.stress_density.max()
        average_density_of_current_stress_episode = episodes_within.stress_density.mean()
        average_deviation_to_daily_mean_current = episodes_within.deviation_to_daily_mean.mean()
    else:
        percentage_of_stress = 0
        maximum_duration_of_current_stress_episode = 0
        average_duration_of_current_stress_episode = 0
        maximum_density_of_current_stress_episode = 0
        average_density_of_current_stress_episode = 0
        average_deviation_to_daily_mean_current = 0

    stress_features = [percentage_of_stress, maximum_duration_of_current_stress_episode,average_duration_of_current_stress_episode,
                       maximum_density_of_current_stress_episode, average_density_of_current_stress_episode,average_deviation_to_daily_mean_current,
                       no_stress_till_now,time_since_last_stress,duration_of_last_stress_episode,average_duration_of_before_stress_episode,
                       density_of_last_stress_episode,average_density_of_before_stress_episode,deviation_to_daily_mean_of_last_stress_episode,
                       percentage_of_stress_before]
    return stress_features




def get_activity_stress_feature(df):
    df = df.sort_values('time').reset_index(drop=True)
    if df.shape[0]<30:
        return pd.DataFrame([],columns=['user','day','time', 'start_time', 'end_time', 'feature','activity','stress','daily'])
    df_activity = activity_data[(activity_data.user==df.user.values[0])&(activity_data.day==df.day.values[0])]
    df_activity['episode_duration'] = df_activity.apply(lambda row:(row['end']-row['start']).total_seconds()/60,axis=1)
    df_activity = df_activity[df_activity['episode_duration']>0]
    df_activity = df_activity.sort_values('start').reset_index(drop=True)
    df_stress = stress_episode_data[(stress_episode_data.user==df.user.values[0])&(stress_episode_data.day==df.day.values[0])]
    df_stress['episode_duration'] = df_stress.apply(lambda row:(row['end']-row['start']).total_seconds()/60,axis=1)
    df_stress = df_stress[(df_stress['episode_duration']>0)&(df_stress['imputed_percentage']<.5)]
    df_stress['episode_label'] = df_stress['stress_episode_label'] 
    df_stress = df_stress.sort_values('start').reset_index(drop=True)
    start_time = min(df_activity['start'].iloc[0],df_activity['start'].iloc[0],df['start_time'].iloc[0]-timedelta(minutes=1))
    end_time = df['end_time'].iloc[df.shape[0]-1]
    all_data = []
    user_id = df.user.values[0]
    day = df.day.values[0]
    for i,row in df.iterrows():
        activity_features = get_activity_features_instance(row,df_activity,start_time,end_time)
        stress_features = get_stress_features_instance(row,df_stress,start_time,end_time)
        all_data.append([user_id,day,row['time'],row['start_time'],row['end_time'],row['feature'],activity_features,stress_features,row['daily']])
    return pd.DataFrame(all_data,columns=['user','day','time', 'start_time', 'end_time', 'feature','activity','stress','daily'])



all_features = ['Time of Day','Gender','Age']
print(len(set(all_features)),len(all_features))
feature_name = ['p90','p95','p80','p5','p10','p20','range_90_to_80','range_20_to_10','range_90_to_10',
                'range_80_to_20','median','iqr','skew']
name = 'current_window_stress'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))
name = 'current_window_stress_diff'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))

all_features += ['time_since_last_visit_smoking_spot','duration_of_stay_in_smoking_spot','fraction_of_minutes_spent_in_smoking_spots_out_of_observed']
print(len(set(all_features)),len(all_features))
name = 'daily_stress'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))
all_features += ['percentage_of_stress', 'maximum_duration_of_current_stress_episode','average_duration_of_current_stress_episode',
               'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
               'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode',
               'density_of_last_stress_episode', 'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode',
               'percentage_of_stress_before']
print(len(set(all_features)),len(all_features))
all_features+= ['percentage_of_active_', 'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode',
                 'no_activity_till_now', 'time_since_last_activity', 'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode',
                 'percentage_of_active_before']
print(len(set(all_features)),len(all_features))
all_features+= ['is_smoking','spread','distance_to_nearest_spot',
                'time_spent_in_transition','time_spent_in_smoking_spot']
print(len(set(all_features)),len(all_features))

3 3
16 16
29 29
32 32
45 45
59 59
67 67
72 72


In [2]:
sliding = 2
intervention_window = 5
data_directory = './data/mRisk/'
demo_label = pd.read_csv(data_directory+'Demographic_Information_CC1_CC2.csv')
demo_label = demo_label.set_index("ID")
location_data_directory = data_directory+'Locations/'
stress_likelihood_data = pickle.load(open(data_directory+'stress_likelihood/rice_1st_version_ecg_rip_imputed_all.p','rb')) 
import warnings
warnings.filterwarnings('ignore')
# obs = 60
# prediction_window = 60
for obs in [10,20,30,45]:
    for prediction_window in [60]:
        data = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_time_and_instantaneous_features)(df) for i,df 
                                                       in stress_likelihood_data.groupby(['user','day'],as_index=False)))
        
        activity_data = pickle.load(open(data_directory+'/activity_episode/activity_episode_data_rice_1st_version_v1.p','rb'))
        activity_data['start'] = activity_data['start'].apply(lambda a:fix_timestamp(a))
        activity_data['end'] = activity_data['end'].apply(lambda a:fix_timestamp(a))
        activity_data['user'] = activity_data['user'].apply(lambda a:user_dict[a] if a in user_dict else a)
        
        stress_episode_data = pickle.load(open(data_directory+'stress_episode/stress_episode_rice_1st_version.p','rb'))
        stress_episode_data['start'] = stress_episode_data['start'].apply(lambda a:fix_timestamp(a))
        stress_episode_data['end'] = stress_episode_data['end'].apply(lambda a:fix_timestamp(a))
        stress_episode_data['user'] = stress_episode_data['user'].apply(lambda a:user_dict[a] if a in user_dict else a)
        
        
        data_final = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_activity_stress_feature)(df) for i,df 
                                                       in data.groupby(['user','day'],as_index=False)))

        data_final = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_activity_episode_encoding)(df) for i,df 
                                                       in data_final.groupby(['user','day'],as_index=False)))

        data_final = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_stress_episode_encoding)(df) for i,df 
                                                       in data_final.groupby(['user','day'],as_index=False)))

        data_final_with_stress_location = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_smoking_status)(df) for i,df 
                                                       in data_final.groupby(['user'],as_index=False)))
        # data_final_with_stress_location_label = data_final_with_stress_location[data_final_with_stress_location.spread>-1]
        data_final_with_stress_location_label = data_final_with_stress_location
        lapse_data = pd.read_csv(data_directory+'mRisk_Lapse_postquit.csv')
        datas = Parallel(n_jobs=-1,verbose=2)(delayed(get_labels)(df) for i,df 
                                                       in data_final_with_stress_location_label.groupby(['user'],as_index=False))
        data_feature_label = pd.concat(datas)
        
        datas = Parallel(n_jobs=-1,verbose=2)(delayed(get_smoking_event_features)(df) for i,df 
                                                       in data_feature_label.groupby(['user'],as_index=False))
        data_feature_label = pd.concat(datas)
        

        data_feature_label['static_features'] = data_feature_label.apply(lambda a:[a['time'].weekday(),
                                                                                   demo_label.loc[np.int64(a['user'])]['Gender'],
                                                                                   demo_label.loc[np.int64(a['user'])]['Age at Enrollment']],axis=1)
        data_feature_label['time_of_day'] = data_feature_label['time'].apply(lambda a:a.hour)
        data_feature_label['time_of_day'] = data_feature_label['time'].apply(lambda a:a.hour)
        data_feature_label = data_feature_label.drop(columns=['Latitude',
                                                               'Longitude', 'is_smoking', 'spread', 'distance_to_nearest_spot',
                                                               'time_spent_in_transition', 'time_spent_in_smoking_spot','s1',
                                                               's2', 's3'],axis=1)
        pickle.dump(data_feature_label,open('./data/parsed_data/episode_encoded_'+str(obs)+'_prediction_'+str(prediction_window)+'_new.p','wb'))
        print('done',obs)
#             lambda a:np.array([a['time'].hour,
#                                demo_label.loc[np.int64(a['user'])]['Gender'],
#                                demo_label.loc[np.int64(a['user'])]['Age at Enrollment']]+list(a['feature'])+list(a['smoking_event_features'])+
#                               list(a['daily'])+list(a['stress'])+list(a['activity'])+list(a['location_features'])).reshape(1,-1),axis=1)
#         for i,f in enumerate(all_features):
#             data_feature_label[f] = data_feature_label['feature_final'].apply(lambda a:a.reshape(-1)[i])
#         pickle.dump(data_feature_label,open('./data/parsed_data/with_non_lapsers_observation_'+str(obs)+'_prediction_'+str(prediction_window)+'_new.p','wb'))
#     print('done',obs)
# # data_feature_label.quit_episode.iloc[30000].shape
# for i,row in data_feature_label.iterrows():
#     if row['quit_episode'].shape!=(30,3):
#         print(row['quit_episode'].shape)
# from collections import Counter
# Counter(data_feature_label.stress_seen_so_far.values)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  3.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   28.2s
[Pa

done 10


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  4.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   25.7s
[Pa

done 20


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   28.5s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  4.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   26.4s
[Pa

done 30


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  4.0min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1114 out of 1114 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 317 tasks      | elapsed:   27.1s
[Pa

done 45


In [4]:
data_feature_label.head()

,user,day,time,start_time,end_time,feature,activity,stress,daily,activity_episode,stress_episode,stress_seen_so_far,location_features,Label,time_since_quit,quit_episode,smoking_event_features,smoking_episode,static_features,time_of_day
0,3002,20170609,2017-06-09 09:23:15.864,2017-06-09 09:01:15.864,2017-06-09 09:46:15.864,"[0.018305571097349527, 0.022517162252416813, 0...","[0.15828, 4.335166666666667, 1.78065, 0, 10.51...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.04166535813304545, 0.08270232549879643, 0.0...","[[[138.0, 1.0, 1.0840500000000002], [139.0, 1....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0,"[0.0, 735.6962251290104, 745.5071316733896, 0....",0,1,"[[1.0, 1.0, 0.0], [2.0, 1.0, 0.0], [3.0, 1.0, ...","[1193.0, 12.268233333333333, 0.012513170321448...","[[1193.0, 1.0, 12.268233333333333], [1194.0, 1...","[4, M, 47.0]",9
1,3002,20170609,2017-06-09 09:25:15.864,2017-06-09 09:03:15.864,2017-06-09 09:48:15.864,"[0.02252713630771877, 0.07917030227698602, 0.0...","[0.15828, 4.335166666666667, 1.78065, 0, 12.51...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.04671717503960372, 0.08754798990416698, 0.0...","[[[140.0, 1.0, 1.0840500000000002], [141.0, 1....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0,"[0.0, 961.0345244043086, 932.6966449024649, 0....",0,3,"[[3.0, 1.0, 0.0], [4.0, 1.0, 0.0], [5.0, 1.0, ...","[1195.0, 12.268233333333333, 0.012504532848505...","[[1195.0, 1.0, 12.268233333333333], [1196.0, 1...","[4, M, 47.0]",9
2,3002,20170609,2017-06-09 09:27:15.864,2017-06-09 09:05:15.864,2017-06-09 09:50:15.864,"[0.07971175761402571, 0.09119545120337831, 0.0...","[0.1789377777777778, 4.335166666666667, 1.6104...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.05115931300792202, 0.08828077594655949, 0.0...","[[[142.0, 1.0, 1.0840500000000002], [143.0, 1....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0,"[0.0, 1212.0778341466164, 1204.561539132832, 0...",0,5,"[[5.0, 1.0, 0.0], [6.0, 1.0, 0.0], [7.0, 1.0, ...","[1197.0, 12.268233333333333, 0.012495907291724...","[[1197.0, 1.0, 12.268233333333333], [1198.0, 1...","[4, M, 47.0]",9
3,3002,20170609,2017-06-09 09:29:15.864,2017-06-09 09:07:15.864,2017-06-09 09:52:15.864,"[0.08756407286063464, 0.09226262340679334, 0.0...","[0.21752925925925928, 4.335166666666667, 1.957...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.05966019346955768, 0.09129900700392053, 0.0...","[[[144.0, 1.0, 1.0840500000000002], [145.0, 1....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0,"[0.0, 1394.1038050236325, 1426.5553689880871, ...",0,7,"[[7.0, 1.0, 0.0], [8.0, 1.0, 0.0], [9.0, 1.0, ...","[1199.0, 12.268233333333333, 0.012487293626461...","[[1199.0, 1.0, 12.268233333333333], [1200.0, 1...","[4, M, 47.0]",9
4,3002,20170609,2017-06-09 09:31:15.864,2017-06-09 09:09:15.864,2017-06-09 09:54:15.864,"[0.08761219032456224, 0.09225490931586838, 0.0...","[0.21752925925925928, 4.335166666666667, 1.957...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.06416907622457085, 0.09110290302760901, 0.0...","[[[146.0, 1.0, 1.0840500000000002], [147.0, 1....","[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0,"[0.0, 1539.122855909688, 1657.4761895015267, 0...",0,9,"[[9.0, 1.0, 0.0], [10.0, 1.0, 0.0], [11.0, 1.0...","[1201.0, 12.268233333333333, 0.012490463006970...","[[1201.0, 1.0, 12.268233333333333], [1202.0, 1...","[4, M, 47.0]",9


In [ ]:
data_feature_label['static_features'] = data_feature_label.apply(lambda a:[a['time'].weekday(),
                                                                                   demo_label.loc[np.int64(a['user'])]['Gender'],
                                                                                   demo_label.loc[np.int64(a['user'])]['Age at Enrollment']],axis=1)
data_feature_label['time_of_day'] = data_feature_label['time'].apply(lambda a:a.hour)
data_feature_label = data_feature_label.drop(columns=['Latitude',
       'Longitude', 'is_smoking', 'spread', 'distance_to_nearest_spot',
       'time_spent_in_transition', 'time_spent_in_smoking_spot','s1',
       's2', 's3'],axis=1)

In [ ]:
pickle.dump(data_feature_label,open('./data/parsed_data/episode_encoded_'+str(obs)+'_prediction_'+str(prediction_window)+'_new.p','wb'))
print('done',obs)

In [ ]:
# stress_episode_data = pickle.load(open(data_directory+'stress_episode/stress_episode_rice_1st_version.p','rb'))
data_feature_label.start_time.iloc[1000].weekday()

In [ ]:
stress_episode_data[stress_likelihood_data]

In [ ]:
data_final_with_stress_location = pd.concat(Parallel(n_jobs=-1,verbose=2)(delayed(get_smoking_status)(df) for i,df 
                                                       in data_final.groupby(['user'],as_index=False)))
data_final_with_stress_location_label = data_final_with_stress_location[data_final_with_stress_location.spread>-1]

In [ ]:
data_feature_label.head()
            
            
#     break

In [ ]:
data_final_with_stress_location_label.shape

In [ ]:
df.head()

In [ ]:
episodes_all

In [ ]:
activity_data[activity_data['episode_label']=='NON STATIONERY']['episode_duration'].quantile(.9)

In [ ]:
data_feature_label.iloc[-1]['feature_final'].shape
X = np.concatenate(list(data_feature_label['feature_final']))

y = data_feature_label['Label'].values
y = np.int64(np.array(y))
y[y>0] = 1
y[y<1] = -1

groups = data_feature_label['user'].values

X1 = np.float64(X[:,2:])

from sklearn import preprocessing
X_time = X[:,0].reshape(-1,1)
X_time = preprocessing.OneHotEncoder().fit_transform(X_time).todense()
X_gender = X[:,1].reshape(-1,1)
X_gender = preprocessing.OneHotEncoder().fit_transform(X_gender).todense()
X = np.concatenate([X_time,X_gender,X1],axis=1)

X.shape,y.shape,groups.shape

In [ ]:
from collections import Counter

In [ ]:
groups1 = groups[y==1]

In [ ]:
np.unique(groups).shape

In [ ]:
all_features = ['Time of Day','Gender','Age']
print(len(set(all_features)),len(all_features))
feature_name = ['p90','p95','p80','p5','p10','p20','range_90_to_80','range_20_to_10','range_90_to_10',
                'range_80_to_20','median','iqr','skew']
name = 'current_window_stress'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))
name = 'current_window_stress_diff'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))

all_features += ['time_since_last_visit_smoking_spot','duration_of_stay_in_smoking_spot','fraction_of_minutes_spent_in_smoking_spots_out_of_observed']
print(len(set(all_features)),len(all_features))
name = 'daily_stress'
all_features += [name+'_'+a for a in feature_name]
print(len(set(all_features)),len(all_features))
all_features += ['percentage_of_stress', 'maximum_duration_of_current_stress_episode','average_duration_of_current_stress_episode',
               'maximum_density_of_current_stress_episode', 'average_density_of_current_stress_episode', 'average_deviation_to_daily_mean_current',
               'no_stress_till_now', 'time_since_last_stress', 'duration_of_last_stress_episode', 'average_duration_of_before_stress_episode',
               'density_of_last_stress_episode', 'average_density_of_before_stress_episode', 'deviation_to_daily_mean_of_last_stress_episode',
               'percentage_of_stress_before']
print(len(set(all_features)),len(all_features))
all_features+= ['percentage_of_active_', 'maximum_duration_of_current_activity_episode', 'average_duration_of_current_window_episode',
                 'no_activity_till_now', 'time_since_last_activity', 'duration_of_last_activity_episode', 'average_duration_of_before_activity_episode',
                 'percentage_of_active_before']
print(len(set(all_features)),len(all_features))
all_features+= ['is_smoking','spread','distance_to_nearest_spot',
                'time_spent_in_transition','time_spent_in_smoking_spot']
print(len(set(all_features)),len(all_features))

In [ ]:
all_features

In [ ]:
for i,f in enumerate(all_features):
    data_feature_label[f] = data_feature_label['feature_final'].apply(lambda a:a.reshape(-1)[i])

In [ ]:
data_feature_label

In [ ]:
name = 'current_window_stress_diff'

In [ ]:
data = pickle.load(open('./data/parsed_data/with_non_lapsers_observation_30_prediction_60.p','rb'))

In [ ]:
df = list(data.groupby(['user'],as_index=False))[0][1]

In [ ]:
data_feature_label.groupby(['user','Label']).count()

In [ ]:
pickle.dump(data_feature_label,open('./data/parsed_data/data_final.p','wb'))
import pickle
import pickle
import numpy as np
import pandas as pd
import os
import zipfile
import shutil
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis
from joblib import Parallel,delayed
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
# print(m/(m+n),'baseline accuracy')
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [ ]:
data_feature_label = pickle.load(open('./data/parsed_data/data_final.p','rb'))

In [ ]:
K = 30

In [ ]:
def get_feature_sequence(df):
    df  = df.sort_values('time').reset_index(drop=True)
    user_id = df.user.values[0]
    all_data = []
    for i in range(K-1,df.shape[0]-1,1):
        row = df.iloc[i]
        features = np.concatenate(list(df['feature_final'].iloc[i-K+1:i+1])).reshape(1,-1)
        label = df['Label'].iloc[i]
        day = df['day'].iloc[i]
        time = df['time'].iloc[i]
        all_data.append([features,user_id,label,day,time])
    return pd.DataFrame(all_data,columns=['features','user','label','day','time'])
        

In [ ]:
sequence_data = data_feature_label.groupby('user',as_index=False).apply(get_feature_sequence)

In [ ]:
X = np.concatenate(list(sequence_data['features']))

y = sequence_data['label'].values

y = np.int64(np.array(y))
y[y>0] = 1
y[y<1] = -1

groups = sequence_data['user'].values

In [ ]:
from collections import Counter

In [ ]:
X.shape,y.shape,groups.shape

In [ ]:
paramGrid = {
            'pca__n_components':[50],
            'rf__min_samples_leaf': [5],
            'rf__max_features': ['auto'],
            'rf__n_estimators': [130,50],
            'rf__n_jobs': [-1],
            'rf__random_state': [42],
            'rf__class_weight':[{-1:w,1:1-w} for w in np.arange(0.01,.1,.02)]
        }

from sklearn.model_selection import train_test_split,LeavePGroupsOut,LeaveOneGroupOut
# gkf = LeavePGroupsOut(n_groups=5)

# for train_index,test_index in gkf.split(X,y,groups=groups):
#     X_train,X_test,y_train,y_test = X[train_index],X[test_index],y[train_index],y[test_index]
#     break
    
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.4,stratify=groups)
clf = Pipeline([('sts',preprocessing.MinMaxScaler()),('pca',PCA()),('rf',RandomForestClassifier())])
gkf = LeaveOneGroupOut()
# list(gkf.split(X,y,groups=groups)
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=gkf.split(X,y,groups=groups),
                           scoring='balanced_accuracy',verbose=1)

grid_search.fit(X,y)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=-1)
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

In [ ]:
data_feature_label.head()

In [ ]:
data_feature_label = pickle.load(open('./data/parsed_data/data_final.p','rb'))
X = np.concatenate(list(data_feature_label['feature_final']))

y = data_feature_label['Label'].values
y = np.int64(np.array(y))
y[y>0] = 1
y[y<1] = -1

groups = data_feature_label['user'].values

X1 =  X[:,1:]

X_time = X[:,0].reshape(-1,1)

X_time = preprocessing.OneHotEncoder().fit_transform(X_time).todense()

X = np.concatenate([X_time,X1],axis=1)

X.shape,y.shape,groups.shape

In [ ]:
paramGrid = {
            'pca__n_components':[7,8,5,6],
            'rf__min_samples_leaf': [5],
            'rf__max_features': ['auto'],
            'rf__n_estimators': [130,50],
            'rf__n_jobs': [-1],
            'rf__random_state': [42],
            'rf__class_weight':[{-1:w,1:1-w} for w in np.arange(0.01,.1,.02)]
        }

from sklearn.model_selection import train_test_split,LeavePGroupsOut,LeaveOneGroupOut
# gkf = LeavePGroupsOut(n_groups=5)

# for train_index,test_index in gkf.split(X,y,groups=groups):
#     X_train,X_test,y_train,y_test = X[train_index],X[test_index],y[train_index],y[test_index]
#     break
    
# X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.4,stratify=groups)
clf = Pipeline([('sts',preprocessing.MinMaxScaler()),('pca',PCA()),('rf',RandomForestClassifier())])
gkf = LeaveOneGroupOut()
# list(gkf.split(X,y,groups=groups)
grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=gkf.split(X,y,groups=groups),
                           scoring='balanced_accuracy',verbose=1)

grid_search.fit(X,y)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X,y,cv=gkf.split(X,y,groups=groups),n_jobs=-1)
print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
clf = grid_search.best_estimator_
y_pred = cross_val_predict(clf,X_train,y_train,cv=gkf.split(X_train,y_train),n_jobs=-1)
print(confusion_matrix(y_train,y_pred),classification_report(y_train,y_pred))

In [ ]:
y_pred_test = clf.fit(X_train,y_train).predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import precision_recall_curve,auc

In [ ]:
p,r,t = precision_recall_curve(y_test,y_pred_test)

In [ ]:
auc(r,p)

In [ ]:
print(classification_report(y_test,clf.fit(X_train,y_train).predict(X_test)))

In [ ]:
clf.fit(X,y)

In [ ]:
pprint(list(10000*clf['rf'].feature_importances_))

In [ ]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

In [ ]:
df = list(data_final_with_stress.groupby('user'))[0][1]

In [ ]:
feature_names  = ['time_of_day','max','p90','p95','p10','p5','mean','slope','iqr','skew','kurtosis']

for i,a in enumerate(feature_names):
    data_final_with_stress[a] = data_final_with_stress['feature'].apply(lambda b:b[i])

activity_feature_names = ['active or not','Time since last activity','duration of last activity episode','Percentage of activity observed',
                          'Average duration of all activity episodes observed','No activity observed till now']

for i,a in enumerate(activity_feature_names):
    data_final_with_stress[a] = data_final_with_stress['activity'].apply(lambda b:b[i])

stress_feature_names = ['within stress or not',
    'Time since last stress episode',
    'duration of last stress episode',
    'density of last stress episode',
    'deviation of amplitude to daily mean for last observed episode',
    'Average duration of all stress episodes observed',
    'Average density of all stress episodes observed',
    'Fraction of minutes observed in stress till now',
    'No stress observed till now']


In [ ]:
for i,a in enumerate(stress_feature_names):
    data_final_with_stress[a] = data_final_with_stress['stress'].apply(lambda b:b[i])

In [ ]:
data_final_with_stress = data_final_with_stress.drop(columns=['feature','stress','activity'],axis=1)
data_final_with_stress.to_csv('./data/parsed_data/feature_stress_activity_feature.csv')

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
import pickle
df = pickle.load(open('./data/lagged_data/obs_30_prediction_60/lagged_20_windows_standardized.p','rb'))

In [ ]:
df.shape

In [1]:
import shutil

In [2]:
shutil.make_archive('./data/mRisk/','zip','./data/mRisk/')

'/home/jupyter/mullah/mrisk/data/mRisk.zip'

In [3]:
import os

In [5]:
os.listdir('./data/mRisk/')

['activity_episode',
 'Mapping.txt',
 'stress_episode',
 'stress_likelihood',
 '.ipynb_checkpoints',
 'Locations.zip',
 'mRisk_lapse_final.csv',
 'Locations',
 'Demographic_Information_CC1_CC2.csv',
 'SmokingSpotStatus',
 'SmokingSpotVisitationHistory_previous',
 'SmokingSpotVisitationHistory',
 'SmokingSpotStatus_previous']